# Quant Course: Lesson 8
## Monte Carlo method, variance reduction

## Martingale Pricing
    
The price of a product with some payoff function $Payoff$ can be calculated as the expected value of the discounted payoff under risk neutral measure. We assume deterministic interest rate so the discount factor can be moved out from the expected value:

$$ V(t) = e^{-r(T-t)}\mathbb{E}[Payoff(S_T)] $$

We will work in the Black-Scholes model where the underlying follows 

$$ dS_t = rS_tdt + \sigma S_tdW_t $$


## Estimating the expected payoff

Law of large numbers: Expected value can be estimated with the sample mean

$$ m = E[X] \approx \frac{1}{N}\sum_iX_i $$ 



$$ V(t) = e^{-r(T-t)}\mathbb{E}[Payoff(S_T)] $$

* Generate n independent samples from $S_T$
* Evaluate the payoff function on the samples and take mean

We implement payoff function for each contract so the second step is easy, the complication arise at the first step.

In the general case we dont know the distribution of $S_T$, we only know it's inital value at some time $t$ and we have an assumption on it's dinamic (our market model). How to obtain samples from $S_T$?

## Euler–Maruyama method

consider the following stochastic diff equation:

$$ dS_t = \mu(t, S_t) dt + \sigma(t,S_t)dW_t $$

One can approximate the solution numerically 

* partition the time interval of $[0,T]$ into $N$ subintervals: $\Delta T = T/N$,   $ \quad t_i = t*\Delta T$
* we know the stock price today $S_0$. We approximate the stock price at the next timepoint:
$$ S_{t_{i+1}} = S_{t_i} + \mu(t_i, S_{t_i}) \Delta T + \sigma(t_i,S_{t_i}) \cdot (W_{t_{i+1}} - W_{t_i}) $$
* $W_t$ is a brownian motion, it's increments are independent normals:
$$ W_{t_{i+1}} - W_{t_i} \sim \mathcal{N}(0,t_{i+1}-t_i) $$
$$ W_{t_{i+1}} - W_{t_i} = \sqrt{\Delta T}\epsilon_i, \quad \epsilon_i\sim \mathcal{N}(0,1)$$

## Solution to the SDE in the BS model

In BS model the risky asset follows GBM:

$$ dS_t = rS_t dt + \sigma S_tdW_t $$

For this relatively simple SDE the solution is known:

$$ S_t = S_0 e^{(r-\frac{\sigma^2}{2})t+\sigma W_t} $$

If we only need samples from $S_T$ we can simulate it directly, no need for partitioning the time interval and simulate the stock price for (from the contract's point of view) useless time points!

## Random number generation

We will use pseudo random numbers: The generated numbers will be statistically random, but if we fix a seed, always the same numbers will be generated

How many random numbers will be needed?

* For a single path, we will need as many $\epsilon$ as the number of simulation tenors.

* Therefor the total number of $\epsilon$ needed is <b>NumberOfPaths * NumberOfSimulTenors<b>

## Simulation tenors

<img src="./img/timegrid.png" style="margin-left:auto; margin-right:auto; width: 1500px;" />

<img src="./img/mc_code_structure.png" style="margin-left:auto; margin-right:auto; width: 1500px;" />

## Excercise 1:

Implement the follwoing methods:
- evolve simulated_spot
- simulate_spot_path
- calc_fair_value

Price a fwd/european option with analytic and MC method

Change the number of simulation paths and see how MC noise changes

In [ ]:
import os 
import sys

sys.path.append("..\..")

from src.pricer import *
MarketData.initialize()

In [ ]:
# import xy

und = Stock.TEST_COMPANY
ls = LongShort.LONG
strike = 1.25
expiry = 1
strike_level = strike * MarketData.get_spot()[und]

model = FlatVolModel(und)

contract_fwd = ForwardContract(und, ls, strike_level, expiry)
contract_opt = EuropeanContract(und, PutCallFwd.PUT, ls, strike_level, expiry)

pricer_fwd_an = ForwardAnalyticPricer(contract_fwd, model, Params())
pricer_opt_an = EuropeanAnalyticPricer(contract_opt, model, Params())

mc_params = MCParams(num_of_path=5000,
                     tenor_frequency=0,
                     antithetic=False,
                     standardize=False,
                     control_variate=False,
                     seed=1)

pricer_fwd_mc = GenericMCPricer(contract_fwd, model, mc_params)
pricer_opt_mc = GenericMCPricer(contract_opt, model, mc_params)

fv_fwd_an = pricer_fwd_an.calc_fair_value()
fv_fwd_mc = pricer_fwd_mc.calc_fair_value()

fv_opt_an = pricer_opt_an.calc_fair_value()
fv_opt_mc = pricer_opt_mc.calc_fair_value()

print("FORWARD AN: " + str(fv_fwd_an))
print("FORWARD MC: " + str(fv_fwd_mc))
print("OPTION AN: " + str(fv_opt_an))
print("OPTION MC: " + str(fv_opt_mc))

In [ ]:
## MC NOISE
fv_list = []
for pathnum in [250, 1000, 5000, 10000]:
    fv_list_for_pathnum = []
    for seed in range(1,11):
        mc_params = MCParams(num_of_path=pathnum,
                             tenor_frequency=0,
                             antithetic=False,
                             standardize=False,
                             control_variate=False,
                             seed=seed)
        pricer_opt_mc = GenericMCPricer(contract_opt, model, mc_params)
        fv_opt_mc = pricer_opt_mc.calc_fair_value()
        fv_list_for_pathnum.append(fv_opt_mc)
    fv_list.append(fv_list_for_pathnum)

#boxplot
import matplotlib.pyplot as plt
 
fig = plt.figure(figsize =(10, 7))
ax = fig.add_axes([0, 0, 1, 1])
bp = ax.boxplot(fv_list)
plt.axhline(y = fv_opt_an, color = 'b', linestyle = '-')
# show plot
plt.show()

# Variance reduction methods

* Standardizing the normal randoms

* Antithetic random numbers 
    
* Control Variate

## Variance Reduction

### Standardizing:

We generate $n$ independent std normals

$$\epsilon = \{\epsilon_1, ..., \epsilon_n\} \qquad \epsilon_i \sim \mathcal{N}(0,1) \, iid$$

The sample itself wont have exactly 0 mean and 1 std dev, so we can adjust it with the sample's mean and std dev:

$$\epsilon^* = \frac{\epsilon - \mu}{\sigma}$$  

### Antithetic randoms:

Once epsilon is generated, flip all the random's sign and reuse them:

$$ \epsilon^* =  \{\epsilon_1, ..., \epsilon_n, -\epsilon_1, ..., -\epsilon_n\} $$


## Variance Reduction, Control Variate

The estimator given control variate Y and parameter b is
$$ X^{CV} = X + b(Y - E[Y]) $$

The new estimator is still unbiased:

$$ E[X^{CV}] = E[X] + b(E[Y] - E[E[Y]]) = E[X] $$

Let's see the variance of the new estimator:

$$ Var[X^{CV}] = Var[X] + Var[b*(Y-E[Y]] + 2Cov[X,b(Y - E[Y] )] = Var[X] + b^2Var[Y] + 2bCov[X,Y] $$

Once we decided what control variate Y we will use, we can choos b such that the modified estimator's variance is minimised:

with $ b^*=-\frac{Cov[X,Y]}{Var[Y]} $, the variance of the new estimator will decrease

$$ Var[X^{CV}] = Var[X] - \frac{(Cov[X,Y])^2}{Var[Y]} $$


## Variance Reduction, Control Variate

$$ X^{CV} = X + b(Y - E[Y])\qquad b^*=-\frac{Cov[X,Y]}{Var[Y]} $$

For example for options one can choose forwards as control variate:

$$ X = (S_T - K)^+ $$
$$ Y = S_T - K $$

1, How to determine $b^*$?

* We will simulate n realization of $S_T$
* Evaluate the payoffs of $X$ and $Y$ on each $S^{(i)}_T$
* Use these samples $[X^{(1)}, ...]$, $[Y^{(1)}, ...]$  to estimate $Cov[X,Y]$ and $Var[Y]$

2, How to determine $E[Y]$?

We can only use such contracts as control variates, which have analytic pricing formula. In that case, we can create an analytic pricer for the contract and invoke it's calc_fair_value method.


## Excercise 2

Implement control variate method:
- apply_control_var_adj
- get_controlvar_helper_pricer

Test the various noise reduction methods

In [ ]:
fv_list = []

numofpath = 5000

mc_params_0 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=False,
                       standardize=False,
                       control_variate=False,
                       seed=1)

mc_params_1 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=True,
                       standardize=False,
                       control_variate=False,
                       seed=1)

mc_params_2 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=False,
                       standardize=True,
                       control_variate=False,
                       seed=1)

mc_params_3 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=False,
                       standardize=False,
                       control_variate=True,
                       seed=1)

mc_params_4 = MCParams(num_of_path=numofpath,
                       tenor_frequency=0,
                       antithetic=True,
                       standardize=True,
                       control_variate=True,
                       seed=1)

fv_list = []
for param in [mc_params_0, mc_params_1, mc_params_2, mc_params_3, mc_params_4]:
    fv_list_for_pathnum = []
    for seed in range(1,11):
        param.seed = seed
        pricer_opt_mc = GenericMCPricer(contract_opt, model, param)
        fv_opt_mc = pricer_opt_mc.calc_fair_value()
        fv_list_for_pathnum.append(fv_opt_mc)
    fv_list.append(fv_list_for_pathnum)

#boxplot
fig = plt.figure(figsize =(10, 7))
ax = fig.add_axes([0, 0, 1, 1])
bp = ax.boxplot(fv_list)
plt.axhline(y = fv_opt_an, color = 'b', linestyle = '-')
# show plot
plt.show()